# Imports

In [195]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import matplotlib.pylab as plt
import numpy as np
import glob as glob
from PIL import Image


# Hyperparameters

In [196]:
batch_size = 128
num_classes = 11
epochs = 12
train_data_dir = 'data/train/*.png'
test_data_dir = 'data/test/*.png'

img_rows, img_cols = 28, 28


# Data preprocessing

In [197]:
(mx_train, my_train), (mx_test, my_test) = mnist.load_data()
x_shape = mx_train.shape

In [198]:
imgs = glob.glob(test_data_dir)

my_x_test = np.zeros(shape = (len(imgs) ,x_shape[1],x_shape[2]))
my_y_test = np.zeros(shape = (len(imgs), ))

for index, img in enumerate(imgs):
    image = Image.open(img).convert('L')
    image = image.resize((img_rows, img_cols), Image.ANTIALIAS)
    arr = np.array(image)
    my_x_test[index] = arr
    my_y_test[index] = -1

In [199]:
imgs = glob.glob(train_data_dir)

my_x_train = np.zeros(shape = (len(imgs) ,x_shape[1],x_shape[2]))
my_y_train = np.zeros(shape = (len(imgs), ))

for index, img in enumerate(imgs):
    image = Image.open(img).convert('L')
    image = image.resize((img_rows, img_cols), Image.ANTIALIAS)
    arr = np.array(image)
    my_x_train[index] = arr
    my_y_train[index] = -1

In [200]:
x_train = np.concatenate((mx_train, my_x_train), axis=0)
y_train = np.concatenate((my_train, my_y_train), axis=0)

x_test = np.concatenate((mx_test, my_x_test), axis=0)
y_test = np.concatenate((my_test, my_y_test), axis=0)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(64833, 28, 28)
(64833,)
(11641, 28, 28)
(11641,)


In [201]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (64833, 28, 28, 1)
64833 train samples
11641 test samples


In [202]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model architecture

In [203]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# Training

In [204]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 64833 samples, validate on 11641 samples
Epoch 1/12
64833/64833 [==============================] - 6s 99us/step - loss: 0.3219 - acc: 0.9021 - val_loss: 0.0715 - val_acc: 0.9768
Epoch 2/12
64833/64833 [==============================] - 6s 93us/step - loss: 0.1133 - acc: 0.9674 - val_loss: 0.0471 - val_acc: 0.9855
Epoch 3/12
64833/64833 [==============================] - 6s 95us/step - loss: 0.0852 - acc: 0.9745 - val_loss: 0.0411 - val_acc: 0.9869
Epoch 4/12
64833/64833 [==============================] - 6s 94us/step - loss: 0.0710 - acc: 0.9798 - val_loss: 0.0386 - val_acc: 0.9871
Epoch 5/12
64833/64833 [==============================] - 6s 96us/step - loss: 0.0620 - acc: 0.9816 - val_loss: 0.0349 - val_acc: 0.9885
Epoch 6/12
64833/64833 [==============================] - 6s 91us/step - loss: 0.0552 - acc: 0.9830 - val_loss: 0.0301 - val_acc: 0.9904
Epoch 7/12
64833/64833 [==============================] - 6s 90us/step - loss: 0.0517 - acc: 0.9851 - val_loss: 0.0304 - val_acc

# Test scores

In [205]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.025223986910340578
Test accuracy: 0.9914096727085302


# Prediction

In [206]:
# load a single image
img = Image.open('data/img/letter_0_8634.png').convert('L')
img = img.resize((img_rows, img_cols), Image.ANTIALIAS)
arr = np.zeros((1,28,28))
arr[0] = np.array(img)
arr = arr.reshape(arr.shape[0], img_rows, img_cols, 1)
# check prediction
pred = model.predict(arr)
print(pred)
classes = pred.argmax(axis=-1)
print(classes)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[10]


# Data augmentation:

In [187]:
datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0,
        horizontal_flip=True,
        fill_mode='nearest')

img = Image.open('data/test/75202.png').convert('L')
img = img.resize((img_rows, img_cols), Image.ANTIALIAS)
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

        
imgs = glob.glob(train_data_dir)

for img in imgs[0:5]:
    image = Image.open(img).convert('L')
    image = image.resize((img_rows, img_cols), Image.ANTIALIAS)
    x = img_to_array(image)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='letter', save_format='png'):
        i += 1
        if i > 5:
            break